<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_Assignment1/blob/main/Assignment2/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv1D,Conv2D, BatchNormalization, Dropout, Activation, Softmax, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam, RMSprop

import numpy as np
import pandas as pd
import os
import random
import pprint

from  matplotlib import pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

In [ ]:
!pip install wandb
!wandb login
import wandb
wandb.init(project="CS6910-assg2", entity="narendv")

In [ ]:
def load_data(dir_train, dir_test, batch):
  
  seed = 42
  
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    samplewise_center = 0,
    horizontal_flip = True,
    rotation_range = 30,
    validation_split = 0.1)
  
  val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    samplewise_center = 0,
    validation_split = 0.1)
  
  test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    samplewise_center = 0)
  
  train_aug_dataset = train_datagen.flow_from_directory(
    dir_train,
    target_size = (img_dim,img_dim),
    batch_size = batch,
    classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia'],
    class_mode='categorical',
    subset = 'training',
    seed = seed)

  train_dataset = val_datagen.flow_from_directory(
    dir_train,
    target_size = (img_dim,img_dim),
    batch_size = batch,
    classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia'],
    class_mode='categorical',
    subset = 'training',
    seed = seed)
  
  val_dataset = val_datagen.flow_from_directory(
    dir_train,
    target_size = (img_dim,img_dim),
    batch_size = batch,
    classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia'],
    class_mode='categorical',
    subset = 'validation',
    seed = seed)
  
  test_dataset = test_datagen.flow_from_directory(
    dir_test,
    targent_size = (img_dim,img_dim),
    batch_size = batch,
    classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia'],
    class_mode='categorical',
    subset = None,
    seed = seed
  )
  
  return train_aug_dataset, train_dataset, val_dataset, test_datagen

In [ ]:
#Do we need this?
learn_rate = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=1000,
    decay_rate=0.9)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
def CNN_wo_BN(filter_matrix, kernel_matrix, activation_matrix, fdropout, dense_no, learning_rate):

  # Input layer
  X_input = keras.Input(shape=(img_dim,img_dim,channel_no))

  # Layer 1
  X = Conv2D(filter_matrix[0], kernel_matrix[0])(X_input)
  X = Activation(activation_matrix[0])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 2
  X = Conv2D(filter_matrix[1], kernel_matrix[1])(X)
  X = Activation(activation_matrix[1])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 3
  X = Conv2D(filter_matrix[2], kernel_matrix[2])(X)
  X = Activation(activation_matrix[2])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 4
  X = Conv2D(filter_matrix[3], kernel_matrix[3])(X)
  X = Activation(activation_matrix[3])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 5
  X = Conv2D(filter_matrix[4], kernel_matrix[4])(X)
  X = Activation(activation_matrix[4])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  X = Flatten()(X)
  X = Dense(dense_no,activation=activation_matrix[5])(X)
  X = Dropout(fdropout)(X)
  X = Dense(10,activation='softmax')(X)
  
  model=Model(inputs=X_input,outputs=X)
  model.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy',  metrics=['accuracy'])

  return model

In [ ]:
def CNN_w_BN(filter_matrix, kernel_matrix, activation_matrix, fdropout, dense_no, learning_rate):

  # Input layer
  X_input = keras.Input(shape=(img_dim,img_dim,channel_no))

  # Layer 1
  X = Conv2D(filter_matrix[0], kernel_matrix[0])(X_input)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[0])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 2
  X = Conv2D(filter_matrix[1], kernel_matrix[1])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[1])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 3
  X = Conv2D(filter_matrix[2], kernel_matrix[2])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[2])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 4
  X = Conv2D(filter_matrix[3], kernel_matrix[3])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[3])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 5
  X = Conv2D(filter_matrix[4], kernel_matrix[4])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[4])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  X = Flatten()(X)
  X = Dense(dense_no,activation=activation_matrix[5])(X)
  X = Dropout(fdropout)(X)
  X = Dense(10,activation='softmax')(X)
  
  model=Model(inputs=X_input,outputs=X)
  model.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy',  metrics=['accuracy'])

  return model

In [ ]:
def model_choose(filter_matrix, kernel_matrix, activation_matrix, dropout, dense_no, batch_norm, learning_rate):
  # if filter_type == 'constant':
  #   filter_matrix = [filter_start]*4
  # elif filter_type == 'doubling':
  #   filter_matrix = [highest_filter/16, highest_filter/8, highest_filter/4, highest_filter/2, highest_filter]
  # elif filter_type == 'halving':
  #   filter_matrix = [highest_filter, highest_filter/2, highest_filter/4, highest_filter/8, highest_filter/16]
  # # wandb.log({"Filters": filter_matrix})
  wandb.log({"Filters": filter_matrix})
  if batch_norm == True:
    model = CNN_w_BN(filter_matrix, kernel_matrix, activation_matrix, dropout, dense_no, learning_rate)
  else:
    model = CNN_wo_BN(filter_matrix, kernel_matrix, activation_matrix, dropout, dense_no, learning_rate)
  
  return model

In [ ]:
@tf.custom_gradient
def GBRelu(x):
  def gradient(dy):
    grad = tf.cast(dy>0,"float32")*tf.cast(x>0, "float32")*dy
    return  grad
  return tf.nn.relu(x), gradient
# Here model is the best model
GBModel = Model(inputs = [model.inputs],outputs = [model.get_layer(index=8).output]) # Here the index is to be checked
layer_dict = [layer for layer in gb_model.layers[1:] if hasattr(layer,'activation')]
for layer in layer_dict:
  if layer.activation == tf.keras.activations.relu:
    layer.activation = GBRelu

with tf.GradientTape() as tape:
  inputs = tf.cast(X_input, tf.float32)
  tape.watch(inputs)
  outputs = GBModel(inputs)

grads = tape.gradient(outputs,inputs)[0]

In [ ]:
sweep_config = {
    'method': 'grid'
    }

parameters_dict = {
    'filter_matrix':{
        'values': [0]
      },
    'kernel_matrix': {
        'values': [0]
      },
    'activation_matrix': {
        'values': [0]
    }
    'learn_rate': {
        'values': [1e-3]
      },
    'epochs': {
        'values': [5,10]
      },
    # 'batch_size': {
    #     'values': [256]
    #   },
    'dropout': {
        'values': [0.2]
      },
    'batch_normalization': {
        'values': [True]
      },
    'dense_number': {
          'values': [512]
      },
    'augmentation':{
        'values' : [True]
    }
}
sweep_config['parameters'] = parameters_dict
pprint.pprint(sweep_config)

def training_sweep(config=None):
    with wandb.init(config=config):
        config = wandb.config

        F = fliters[config.filter_matrix]
        K = kernels[config.kernel_matrix]
        A = activations[config.activation_matrix]

        model = model_choose(F, K, A, config.dropout, config.dense_number, config.batch_normalization, config.learn_rate)
        if config.augmentation == True:
          train = train_aug_dataset
        else:
          train = train_dataset

        history = model.fit(train, 
                            epochs=config.epochs,
                            validation_data = val,
                            callbacks = callback
                            )
        wandb.log(history.history)

In [ ]:
global img_dim
img_dim = 150

global channel_no
channel_no = 3 #3 for RGB images, 1 for greyscale

global batch
batch = 256

global filters
filters = [[64,128,256,512,1024]]

global kernels
kernels = [[(3,3)]*5]

global activations
activations = [['relu']*6]

global F, global K, global A

global train_aug_dataset, global train_dataset, global val_dataset, global test_datagen
train_aug_dataset, train_dataset, val_dataset, test_datagen = load_data('/content/drive/MyDrive/inaturalist_12K/train', '/content/drive/MyDrive/inaturalist_12K/val', batch)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="CS6910-assg2")
wandb.agent(sweep_id, training_sweep)